In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, f1_score
import time

# Load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

total_vram_usage = 0
total_f1 = 0
num_checks = 5

for epoch in range(num_checks):
    start_time = time.time()

    with tf.device('/GPU:0'):
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=1, verbose=2)

        y_pred = model.predict(x_test)
        y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

        precision = precision_score(y_test, y_pred_classes, average='weighted')
        recall = recall_score(y_test, y_pred_classes, average='weighted')
        f1 = f1_score(y_test, y_pred_classes, average='weighted')

        print(f"Neural Network Model, Epoch {epoch + 1}/5 - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    end_time = time.time()

    gpu_info = !nvidia-smi
    vram_usage_line = next((line for line in gpu_info if 'MiB /' in line), None)

    if vram_usage_line:
        vram_used_str = vram_usage_line.split()[8]
        vram_used = int(vram_used_str[:-3])

        print(f"VRAM Usage: {vram_used} MiB")

        total_vram_usage += vram_used
        total_f1 += f1

        time.sleep(5)

average_vram_usage = total_vram_usage / num_checks
average_f1 = total_f1 / num_checks
print(f"Average VRAM Usage: {average_vram_usage} MiB")
print(f"Average F1 Score: {round(average_f1, 2)}")


11490434/11490434 [==============================] - 0s 0us/step
1 Physical GPUs, 1 Logical GPUs
1875/1875 - 15s - loss: 0.2581 - accuracy: 0.9261 - 15s/epoch - 8ms/step
313/313 [==============================] - 1s 2ms/step
Neural Network Model, Epoch 1/5 - Precision: 0.9590, Recall: 0.9589, F1 Score: 0.9589
VRAM Usage: 639 MiB
1875/1875 - 4s - loss: 0.1133 - accuracy: 0.9663 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 2ms/step
Neural Network Model, Epoch 2/5 - Precision: 0.9719, Recall: 0.9718, F1 Score: 0.9718
VRAM Usage: 1151 MiB
1875/1875 - 4s - loss: 0.0784 - accuracy: 0.9763 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 3ms/step
Neural Network Model, Epoch 3/5 - Precision: 0.9747, Recall: 0.9746, F1 Score: 0.9746
VRAM Usage: 1151 MiB
1875/1875 - 4s - loss: 0.0584 - accuracy: 0.9815 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 2ms/step
Neural Network Model, Epoch 4/5 - Precision: 0.9766, Recall: 0.9766, F1 Score